In [1]:
import os
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import getpass
import glob
import seaborn as sns
import functions
import lfp_pre_processing_functions
import power_functions
import coherence_functions
import spectrogram_plotting_functions
import plotting_styles
import scipy.stats
import mne_connectivity
importlib.reload(functions) #loads our custom made functions.py file
importlib.reload(spectrogram_plotting_functions)
importlib.reload(plotting_styles)

linestyle = plotting_styles.linestyles
colors = plotting_styles.colors

In [32]:
%pip install mne-qt-browser

import mne
print(mne.get_config())  # same as mne.get_config(key=None)
mne.set_config('MNE_BROWSER_BACKEND', 'qt')  # set the backend to matplotlib

   ---------------------------------------- 0.0/76.1 kB ? eta -:--:--
   ------------------------------------- -- 71.7/76.1 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 76.1/76.1 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   --------- ------------------------------ 0.5/1.9 MB 9.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.1/1.9 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------  1.9/1.9 MB 13.6 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 12.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/871.8 kB ? eta -:--:--
   --------------------------------------  870.4/871.8 kB 18.7 MB/s eta 0:00:01
   --------------------------------------- 871.8/871.8 kB 13.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/95.0 kB ? eta -:--:--
   ---------------------------------------- 95.0/95.0 kB 5.7 MB/s eta 0:00:00
Note: you may need 

Channels marked as bad:
none
Channels marked as bad:
none


In [2]:
#Fetch the current user
user= (getpass.getuser())
print("Hello", user)

#Set the basepath, savepath and load the data files
base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
files = glob.glob(base+'\\all_data_mat\\*.mat')
savepath = base+'\\results\\'
print(files)

all_bands_dict = {'total':[1,100], 'beta':[12,30], 'gamma':[30,80], 'theta':[4,12]}


Hello Thomas
['C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230529_dk1_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230529_dk3_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230529_dk5_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230529_dk6_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230531_dk1_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230531_dk3_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230531_dk5_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230531_dk6_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230609_dk1_BW_nocontext_day1.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230609_dk3_BW_nocontext_day1.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230610_dk1_BW_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat\\20230610_dk3_BW_noconte

In [33]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
import mne
import mne_connectivity
files=[f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat\\20230616_dk6_BW_context_day2.mat']
event_data_df=[]
con_data_df=[]

con_data_df_shuffled=[]
shuffled_event_data_df=[]
events_codes_all = {}
random_baseline_data=[]
baseline_lfp_all=[]

for file_num,file in enumerate(files):
    #if 'dk1' in file:
        
        #print(file)
        base_name = os.path.basename(file)
        base_name, _ = os.path.splitext(base_name)

        date, rat_id, task = lfp_pre_processing_functions.exp_params(base_name)
        print(date, rat_id, task)
        if task == 'nocontextday2' or task == 'nocontextos2':
            task = 'nocontext'
        if task =='nocontext':
            continue
        if rat_id=='dk3':
            continue
        f = h5py.File(file, 'r')
        channels = list(f.keys())
        print(channels)
         
        if not any("AON" in channel or "vHp" in channel for channel in channels):
            print("No AON or vHp channels in this file")
            continue

        events,reference_electrode=lfp_pre_processing_functions.get_keyboard_and_ref_channels(f,channels)

    #finding global start and end time of all channels, since they start and end recordings at different times
        global_start_time, global_end_time=lfp_pre_processing_functions.find_global_start_end_times(f,channels)
        
        ## Reference electrode finding and padding
        reference_time = np.array(reference_electrode['times']).flatten()
        reference_value = np.array(reference_electrode['values']).flatten()
        padd_ref_data,padded_ref_time=lfp_pre_processing_functions.pad_raw_data_raw_time(reference_value,reference_time,global_start_time,global_end_time,sampling_rate=2000)

        events_codes = np.array(events['codes'][0])
        events_times = np.array(events['times'][0])
        print(events_codes.shape, events_times.shape, 'Keyboard')
        print(events_codes)
        events_codes_all[base_name] = events_codes
        print(events_codes_all)
        epochs = lfp_pre_processing_functions.generate_epochs_with_first_event(events_codes, events_times)
        #epochs = functions.generate_specific_num_of_epochs_with_first_event(events_codes, events_times,5)
        aon_lfp_channels=[x for x in channels if 'AON' in x ]
        vHp_lfp_channels=[x for x in channels if 'vHp' in x ]
        data_channels=np.concatenate((aon_lfp_channels,vHp_lfp_channels))
        #print(all_channels)
        annotations = mne.Annotations(onset=events_times, duration=np.zeros_like(events_times), description=events_codes)
        keyboard_dict={'98':'b','119':'w','49':'1','48':'0'}
        annotations.rename(keyboard_dict)
        
        all_channels = list(data_channels) + ['reference']
        info = mne.create_info(ch_names=list(all_channels), sfreq=2000, ch_types='eeg')

        all_channels_data= []

        first_event = events_times[0]
        raw_data = []
        for channel_num,channeli in enumerate(data_channels):
            if "AON" in channeli or "vHp" in channeli:
                channel_id = channeli
                data_all = f[channeli]
                raw_data = np.array(data_all['values']).flatten()
                raw_time = np.array(data_all['times']).flatten()

                padded_data,padded_time=lfp_pre_processing_functions.pad_raw_data_raw_time(raw_data,raw_time,global_start_time,global_end_time,2000)

                print(len(padded_data), len(padded_data), channel_id)
                all_channels_data.append(padded_data)
            
        all_channels_data = np.array(all_channels_data)
        all_channels_data = np.vstack((all_channels_data, padd_ref_data))
        print(all_channels_data.shape)
        mne_all_channels_data = mne.io.RawArray(all_channels_data, info)
        mne_all_channels_data.set_eeg_reference(ref_channels=['reference'], projection=False)
        mne_all_channels_data.set_annotations(annotations)
mne.viz.plot_raw(mne_all_channels_data, duration=60, scalings='auto', show=True, block=True)
        #                 sampling_rate = int(1 / data_all['interval'][0][0])
#                 #print(raw_data.shape, raw_time.shape, sampling_rate)
#                 subtracted_data = padded_data - padd_ref_data
#                 raw_data=subtracted_data
#                 notch_filtered_data = lfp_pre_processing_functions.iir_notch(raw_data, sampling_rate, 60)

#                 data_before, time, baseline_mean, baseline_std=lfp_pre_processing_functions.baseline_data_normalization(notch_filtered_data, raw_time, first_event, sampling_rate)
#                 first_event_index=np.where(raw_time>first_event)[0][0]

#                 mne_baseline_data[0,channel_num,:]=list(data_before)
#                 mne_baseline_data_shuffled[0,channel_num,:]=list(np.random.permutation(data_before))
#                 total = notch_filtered_data

                
#                 for i, epochi in enumerate(epochs):
#                     door_timestamp = epochi[0][0]
#                     trial_type = epochi[0][1]
#                     dig_type = epochi[1, 1]
#                     #print(dig_type)
#                     dig_timestamp = epochi[1, 0]
#                     #print(door_timestamp, trial_type, dig_timestamp, dig_type)
#                     data_trial_before, data_trial_after=lfp_pre_processing_functions.extract_event_data(notch_filtered_data,time,door_timestamp,sampling_rate,0.7)
#                     data_dig_before, data_dig_after=lfp_pre_processing_functions.extract_event_data(notch_filtered_data,time,dig_timestamp,sampling_rate,0.7)
#                     data_around_door=np.concatenate((data_trial_before, data_trial_after))
#                     data_around_dig=np.concatenate((data_dig_before, data_dig_after))

#                     epoch_data = [data_trial_before, data_trial_after, data_dig_before, data_dig_after, data_around_door, data_around_dig]
#                     event_data_list = [lfp_pre_processing_functions.zscore_event_data(x, baseline_std) for x in epoch_data]

#                     mne_epoch_door_before[i,channel_num,:]=list(event_data_list[0])
#                     mne_epoch_door_after[i,channel_num,:]=list(event_data_list[1])
#                     mne_epoch_dig_before[i,channel_num,:]=list(event_data_list[2])
#                     mne_epoch_dig_after[i,channel_num,:]=list(event_data_list[3])
#                     mne_epoch_around_door[i,channel_num,:]=list(event_data_list[4])
#                     mne_epoch_around_dig[i,channel_num,:]=list(event_data_list[5])

#                     mne_epoch_door_before_shuffled[i,channel_num,:]=list(np.random.permutation(event_data_list[0]))
#                     mne_epoch_door_after_shuffled[i,channel_num,:]=list(np.random.permutation(event_data_list[1]))
#                     mne_epoch_dig_before_shuffled[i,channel_num,:]=list(np.random.permutation(event_data_list[2]))
#                     mne_epoch_dig_after_shuffled[i,channel_num,:]=list(np.random.permutation(event_data_list[3]))
#                     mne_epoch_around_door_shuffled[i,channel_num,:]=list(np.random.permutation(event_data_list[4]))
#                     mne_epoch_around_dig_shuffled[i,channel_num,:]=list(np.random.permutation(event_data_list[5]))

#         if len(all_channels)>0:
#             fs=2000
#             freqs = np.arange(1,100)
#             n_cycles = freqs/3
#             info = mne.create_info(ch_names=list(all_channels), sfreq=fs, ch_types='eeg')
#             mne_baseline = mne.EpochsArray(mne_baseline_data, info)
#             mne_epoch_door_before = mne.EpochsArray(mne_epoch_door_before, info)
#             mne_epoch_door_after= mne.EpochsArray(mne_epoch_door_after, info)
#             mne_epoch_dig_before = mne.EpochsArray(mne_epoch_dig_before, info)
#             mne_epoch_dig_after = mne.EpochsArray(mne_epoch_dig_after, info)
#             mne_epoch_around_door = mne.EpochsArray(mne_epoch_around_door, info)
#             mne_epoch_around_dig = mne.EpochsArray(mne_epoch_around_dig, info)
            
#             row_list=[file_num,rat_id,task,mne_baseline,mne_epoch_door_before,mne_epoch_door_after,mne_epoch_dig_before,mne_epoch_dig_after,mne_epoch_around_door,mne_epoch_around_dig]
            
#             mne_baseline_shuffled = mne.EpochsArray(mne_baseline_data_shuffled, info)
#             mne_epoch_door_before_shuffled = mne.EpochsArray(mne_epoch_door_before_shuffled, info)
#             mne_epoch_door_after_shuffled = mne.EpochsArray(mne_epoch_door_after_shuffled, info)
#             mne_epoch_dig_before_shuffled = mne.EpochsArray(mne_epoch_dig_before_shuffled, info)
#             mne_epoch_dig_after_shuffled = mne.EpochsArray(mne_epoch_dig_after_shuffled, info)
#             mne_epoch_around_door_shuffled = mne.EpochsArray(mne_epoch_around_door_shuffled, info)
#             mne_epoch_around_dig_shuffled = mne.EpochsArray(mne_epoch_around_dig_shuffled, info)
#             row_list_shuffled=[file_num,rat_id,task,mne_baseline_shuffled,mne_epoch_door_before_shuffled,mne_epoch_door_after_shuffled,mne_epoch_dig_before_shuffled,mne_epoch_dig_after_shuffled,mne_epoch_around_door_shuffled,mne_epoch_around_dig_shuffled]
#             shuffled_event_data_df.append(row_list_shuffled)

#             con_data_df.append(row_list)
#             con_data_df_shuffled.append(row_list_shuffled)


# con_data_df=pd.DataFrame(con_data_df, columns=['experiment','rat_id','task','mne_baseline','mne_epoch_door_before','mne_epoch_door_after','mne_epoch_dig_before','mne_epoch_dig_after','mne_epoch_around_door','mne_epoch_around_dig'])
# con_data_df.to_pickle(savepath+'mne_epochs_array_df_truncated.pkl')

# con_data_df_shuffled=pd.DataFrame(con_data_df_shuffled, columns=['experiment','rat_id','task','mne_baseline','mne_epoch_door_before','mne_epoch_door_after','mne_epoch_dig_before','mne_epoch_dig_after','mne_epoch_around_door','mne_epoch_around_dig'])
# con_data_df_shuffled.to_pickle(savepath+'mne_epochs_array_df_shuffled_truncated.pkl')


20230616 dk6 BWcontext
['Keyboard', 'LFP1_vHp', 'LFP2_vHp', 'LFP3_AON', 'LFP4_AON', 'Ref', 'Respirat', 'file']
(83,) (83,) Keyboard
[119  48  49  98  49  98  49 119  49  98  49 119  49 119  49 119  49  98
  49  98  48  49 119  49 119  49 119  49 119  49  98  48  49 119  49 119
  49  98  49  98  49 119  49 119  49  98  49  98  49  98  49  98  49 119
  49 119  49  98  49  98  49  98  49 119  49  98  49 119  49  98  49 119
  49  98  49 119  49  98  49  98  49 119  49]
{'20230616_dk6_BW_context_day2': array([119,  48,  49,  98,  49,  98,  49, 119,  49,  98,  49, 119,  49,
       119,  49, 119,  49,  98,  49,  98,  48,  49, 119,  49, 119,  49,
       119,  49, 119,  49,  98,  48,  49, 119,  49, 119,  49,  98,  49,
        98,  49, 119,  49, 119,  49,  98,  49,  98,  49,  98,  49,  98,
        49, 119,  49, 119,  49,  98,  49,  98,  49,  98,  49, 119,  49,
        98,  49, 119,  49,  98,  49, 119,  49,  98,  49, 119,  49,  98,
        49,  98,  49, 119,  49], dtype=uint8)}
2211369 2211369 LF